## **<font color="red">State</font>**
- **Bypassing State Injection with `InstructionProvider`:**
  - In some cases, you might want to use `{{ and }}` literally in your instructions without triggering the state injection mechanism. For example, you might be writing instructions for an agent that helps with a templating language that uses the same syntax.
  - To achieve this, you can provide a function to the `instruction` parameter instead of a string. This function is called an `InstructionProvider`. When you use an InstructionProvider, the ADK will not attempt to inject state, and your instruction string will be passed to the model as-is.
  - The `InstructionProvider` function receives a ReadonlyContext object, which you can use to access session state or other contextual information if you need to build the instruction dynamically.

In [1]:
import os
from config import config
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# This is an InstrucitonProvider
def custom_instruction_provider(context: ReadonlyContext)-> str:
    # You can optionally use the context to build the instruction
    # For this example, we'll return a static string with literal braces.
    return "This is an instruction with {{literal_braces}} that will not be replaced."\
    

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction=custom_instruction_provider
)




- If you want to both use an `InstructionProvider` and inject state into your instructions, you can use the `inject_session_state` utility function.
- **Benefits of Direct Injection:**
  - ***Clarity:** Makes it explicit which parts of the instruction are dynamic and based on session state.
  - **Reliability:** Avoids relying on the LLM to correctly interpret natural language instructions to access state.
  - **Maintainability:** Simplifies instruction strings and reduces the risk of errors when updating state variable names.
- **Relation to Other State Access Methods**
  - This direct injection method is specific to LlmAgent instructions. Refers to the following seciton for more information on other state access methods.

In [2]:
import os
from config import config
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.utils import instructions_utils

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# Dynamic Context Provider
async def my_dynamic_instruction_provider(context: ReadonlyContext) -> str:
    template = "This is a {adjective} instruction with {{literal_braces}}."
    # This will inject the 'adjective' state variable but leave the literal braces.
    return await instructions_utils.inject_session_state(template, context)

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction=my_dynamic_instruction_provider
)




- **The Right Way to Modify State:**
  - When you need to change the session state, the correct and safest method is to *directly modify the `state` object on the** `Context` provided to your function (e.g., `callback_context.state['my_key'] = 'new_value'`). This is considered "direct state manipulation" in the right way, as the framework automatically tracks these changes.
  - This is critically different from directly modifying the `state` on a `Session` object you retrieve from the `SessionService` (e.g., `my_session.state['my_key'] = 'new_value'`). *You should avoid this*, as it bypasses the ADK's event tracking and can lead to lost data. The "Warning" section at the end of this page has more details on this important distinction.
- State should always be updated as part of adding an Event to the session history using `session_service.append_event()`. This ensures changes are tracked, persistence works correctly, and updates are thread-safe.

#### **1. The Easy Way: `output_key` (for Agent Text Responses)**
- This is the simplest method for saving and agent's final text response directly into the state. When defining your `LlmAgent`, specify me `output_key`:

In [3]:
import os
from config import config
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# Agent
greeting_agent = LlmAgent(
    name="Greeter",
    model="gemini-2.5-flash",
    instruction="Generate a short, friendly greeting",
    output_key="last_greeting" # Save response to state['last_greeting']
)

# --- Setup Runner and Session ---
APP_NAME, USER_ID, SESSION_ID = "state_app", "user1", "session1"

# Session: Memory
session_service = InMemorySessionService()

# Runner: AgentExecutor
runner = Runner(
    agent=greeting_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Create Session: Memory Initializaiton
session = await session_service.create_session(app_name=APP_NAME, 
                                               user_id=USER_ID,
                                               session_id=SESSION_ID)
print(f"Initial state: {session.state}")

# --- Run the Agent ---
# Runner handles calling append_event, which uses the output_key to automatically create the state_delta.
user_message = Content(parts=[Part(text="Hello")])  # PromptTemplate

for event in runner.run(user_id=USER_ID,
                        session_id=SESSION_ID,
                        new_message=user_message):
    if event.is_final_response():
        print(f"Agent Response.")

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=APP_NAME, 
                                                    user_id=USER_ID,
                                                    session_id=SESSION_ID)
print(f"State after agent run: {updated_session.state}")


Initial state: {}
Agent Response.
State after agent run: {'last_greeting': 'Hi there!'}


In [4]:
# --- Run the Agent ---
# Runner handles calling append_event, which uses the output_key to automatically create the state_delta.
user_message = Content(parts=[Part(text="What is thermodynamic?")])  # PromptTemplate

for event in runner.run(user_id=USER_ID,
                        session_id=SESSION_ID,
                        new_message=user_message):
    if event.is_final_response():
        print(f"Agent Response.")

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=APP_NAME, 
                                                    user_id=USER_ID,
                                                    session_id=SESSION_ID)
print(f"State after agent run: {updated_session.state}")


Agent Response.
State after agent run: {'last_greeting': "Thermodynamics is a branch of physics that deals with **heat and its relation to other forms of energy and work.**\n\nIn simpler terms, it's the study of how energy is transferred, transformed, and conserved in physical systems, especially focusing on phenomena like:\n*   **Heat:** The transfer of thermal energy.\n*   **Work:** Energy transferred by a force acting over a distance.\n*   **Temperature:** A measure of the average kinetic energy of particles.\n*   **Pressure:** Force per unit area.\n*   **Volume:** The amount of space occupied.\n*   **Entropy:** A measure of disorder or randomness in a system.\n\nIt's governed by a set of fundamental laws (the four laws of thermodynamics) that describe how energy behaves and places limits on what's possible in energy conversion. Thermodynamics is crucial for understanding everything from how engines work to the processes in chemical reactions and biological systems."}


- Behind the scenes, the `Runner` uses the `output_key` to create the necessary `EventActions` with a `state_delta` and calls `append_event`.

#### **2. Then Standard Way: `EventActions.state_delta` (for Complex Updates)**
- For more complex scenarios (updating multiple keys, non-string values, specific scopes like `user:` or `app:`, or updates not tried directly to the agent's final text), you manually construct the `state_delta` within `EventActions` .

In [9]:
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event, EventActions
from google.genai.types import Part, Content
import time

# --- Setup ---
session_service = InMemorySessionService()

APP_NAME, USER_ID, SESSION_ID = "state_app_manual", "user2", "session2"

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"user:login_count": 0, "task_status": "idle"}
)
print(f"Initial state: {session.state}")

# --- Define State Changes ---
current_time = time.time()
state_changes = {
    "task_status": "active",  # Update session state
    "user:login_count": session.state.get("user:login_count", 0) + 1, # Update user state
    "user:last_login_ts": current_time, # Add user state
    "temp:validation_needed": True, # Add temporary state (will be discarded)
}

# --- Create Event with Actions ---
actions_with_update = EventActions(state_delta=state_changes)
# This event might represent an internal system action, not just an agent response
system_event = Event(
    invocation_id= "inv_login_update",
    author="system", # Or 'agent', 'tool' etc.
    actions=actions_with_update,
    timestamp=current_time
    # content might be None or represent the action taken
)

# --- Append the Event (This updates the state) ---
await session_service.append_event(session, system_event)
print("`append_event` called with explicit state delta.")

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=APP_NAME,
                                                    user_id=USER_ID,
                                                    session_id=SESSION_ID
                                                   )
print(f"State after event: {updated_session.state}")


Initial state: {'task_status': 'idle', 'user:login_count': 0}
`append_event` called with explicit state delta.
State after event: {'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1770985676.3582003}


In [5]:
import os
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from config import config

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "story_app"
USER_ID = "user_1"
SESSION_ID = "session_1"

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction="Write a short story about a cat."
)



session_service = InMemorySessionService()

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    agent=story_generator,
    session_service=session_service,
    app_name=APP_NAME,
)

event_generator = runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.UserContent(
        parts=[types.Part(text="Write about friendship")]
    ),
)

for event in event_generator:
    if event.content and event.content.parts:
        print(event.content.parts[0].text)


Luna was a shadow cat, sleek and black, with eyes like chips of emerald. For months, she’d been a silent fixture around Mrs. Gable’s small cottage, observing the world with a cool, detached curiosity. She knew the best sunbeam on the porch, the softest patch of long grass by the rose bushes, and the exact time Mrs. Gable put out a fresh bowl of water near the birdbath.

Mrs. Gable, a woman whose wrinkles held the stories of many years, never tried to rush Luna. She’d simply talk to her from a distance, her voice a soft murmur that never startled the cautious cat. "Hello there, midnight kitty," she'd say, placing a small bowl of kibble on the porch steps before retreating. Luna would wait until the door clicked shut, then pad forward, her movements fluid and silent, to eat.

Their friendship began with this quiet ritual – a space shared, an offering made, and an acceptance given. Luna would gradually spend more time on the porch, first just outside the circle of light from the window, t

In [6]:
import os
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from config import config

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "story_app"
USER_ID = "user_1"
SESSION_ID = "session_1"

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction="Write a short story about a cat."
)

async def main():

    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

    runner = Runner(
        agent=story_generator,
        session_service=session_service,
        app_name=APP_NAME,
    )

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=types.UserContent(
            parts=[types.Part(text="Write about friendship")]
        ),
    ):
        if event.content and event.content.parts:
            print(event.content.parts[0].text)

await main()


Luna was a creature of quiet habits and even quieter purrs. Her fur, the color of twilight, absorbed the morning sun and melted into the evening shadows. Her days revolved around sunbeams, the precise moment breakfast appeared, and the rhythmic sound of Mrs. Gable's knitting needles.

Mrs. Gable was Luna’s human, a woman whose hands were gnarled with age but whose touch was always feather-light and infinitely gentle. Their friendship wasn't one of boisterous games or loud declarations. It was built on shared silence, on the warmth of a lap, and the unspoken understanding that bloomed between them.

One crisp autumn afternoon, a chill had seeped into the old house, and a deeper chill seemed to have settled into Mrs. Gable's spirit. She sat in her armchair by the window, not knitting, not reading, just gazing out at the rustling, falling leaves. Her shoulders were hunched, and a sigh, barely audible, escaped her lips.

Luna, who had been curled into a tight, black knot on the worn Persia